In [ ]:
'''2. output_file:
   {
   "id": "d3b479933e716fb388dfb297e881054c",
   "question": {
      "stem": "If a lantern is not for sale, where is it likely to be?"
      "choices": [{"label": "A", "text": "antique shop"}, {"label": "B", "text": "house"}, {"label": "C", "text": "dark place"}]
    },
    "answerKey":"B",

    "statements":[
        {label:true, stem: "If a lantern is not for sale, it likely to be at house"},
        {label:false, stem: "If a lantern is not for sale, it likely to be at antique shop"},
        {label:false, stem: "If a lantern is not for sale, it likely to be at dark place"}
        ]
  }'''

In [ ]:
import json

In [ ]:
def clean_riddle(riddle):
    riddle_clean = riddle.split('（')
    if '打一' in riddle_clean[1]:
        raw_key = riddle_clean[1][2:-1]
    elif '打' in riddle_clean[1][:1]:
        raw_key = riddle_clean[1][1:-1]
        if '一' in riddle_clean[1][-2:]:
            raw_key = riddle_clean[1][1:-2]
    else:
        raw_key = riddle_clean[1][:-1]
    
    key = raw_key
    
    if raw_key == '物':
        key = '东西'
    if raw_key[-1] == '名':
        key = raw_key[:-1]
    
    prompt = riddle_clean[0] + '这个' + key + '是'
    return prompt

In [ ]:
# Create the output json dictionary from the input json, premise and hypothesis statement
def create_output_dict(input_json: dict, statement: str, label: bool) -> dict:
    if "statements" not in input_json:
        input_json["statements"] = []
    input_json["statements"].append({"label": label, "statement": statement})
    return input_json

In [ ]:
def create_hypothesis(question, choice_text):
    return question + choice_text

In [ ]:
# Convert the QA file json to output dictionary containing premise and hypothesis
def convert_qa_to_entailment(qa_index, qa_row):
    qa_dict = {}
    question_text = clean_riddle(qa_row["riddle"])
    choices = [qa_row["choice" + str(i)] for i in range(5)]
    choices_id = [i for i in range(5)]
    true_ans = qa_row["choice" + str(qa_row['label'])]
    qid = qa_index
    qa_dict['id'] = qid
    qa_dict['question'] = {'stem': qa_row['riddle'], 'choices':[{'label' : str(choices_id[i]), 'text':choices[i]} for i in range(5)]}
    qa_dict['answerKey'] = str(qa_row['label'])
    for choice, cid in zip(choices, choices_id):
        choice_text = choice
        statement = create_hypothesis(question_text, choice_text)
        create_output_dict(qa_dict, statement, cid == int(qa_row['label']))
    return qa_dict

In [ ]:
def convert_to_entailment(qa_df: str, output_file: str, ans_pos: bool=False):
    print(f'converting {qa_df} to entailment dataset...')
    # 读取数据集
    # nrow代表数据集数据条目数
    with open(output_file, 'w') as output_handle:
        for index, row in qa_df.iterrows():
            # print("Writing to {} from {}".format(output_file, qa_file))
            output_dict = convert_qa_to_entailment(index, row)
            output_handle.write(json.dumps(output_dict, ensure_ascii=False))
            output_handle.write("\n")
    print(f'converted statements saved to {output_file}')
    print()

In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/littlesoldierproject-data/train_riddle.csv')
val_df = pd.read_csv('../input/littlesoldierproject-data/val_riddle.csv')
convert_to_entailment(train_df, 'train.statement_zh.jsonl')
convert_to_entailment(val_df, 'val.statement_zh.jsonl')